## Data analysis with SQL

I recently found my old notes for SQL. I switched a long time ago to Pandas, which is much more modern and advanced tool. However, SQL is still widely used in the IT world.
I love Pandas, but the simplicity of the SQL strikes me.<br>
There is small part of sales data analysis.

--------------------------------------------------------------------------------------

```sql
SELECT  s.sales_date,
        s.order_id,
        s.product_id,
        s.customer_id,
        s.total_amount,
        savg.avg_total_amount	
FROM sales s,
        ( SELECT AVG (total_amount) AS avg_total_amount FROM sales ) savg
```
- added new column - average total sales.



```sql
SELECT  s.sales_date,
        s.order_id, 
        s.product_id, 	
        s.customer_id, 
        s.total_amount, 
        savg.avg_total_amount
FROM sales s,
        ( SELECT sales_date,
AVG (total_amount) AS avg_total_amount FROM sales
GROUP BY sales_date) savg
WHERE s.sales_date = savg.sales_date
ORDER BY (1)
```
- added another new column with average daily sales.



```sql
SELECT  s.sales_date, 
        s.order_id, 
        s.product_id, 
        s.customer_id, 
        s.total_amount, 
        AVG (total_amount)  over (partition by sales_date ) AS avg_total_amount
FROM sales s
```
- added new column with average daily sales — this is easy syntax!



```sql
SELECT  s.sales_date, 
        s.order_id, 
        s.product_id, 
        s.customer_id, 
        s.total_amount, 
        AVG (total_amount)  over () AS avg_total_amount
FROM sales s
```
- new average (total) sales column added



```sql
SELECT  sales_date, 
        order_id, 
        product_id, 
        sales_amount, 
        avg(sales_amount) over () 
FROM sales
```
- after daily sales column, we added avg(sales_amount) column



```sql
SELECT  sales_date, 
        order_id, 
        product_id, 
        sales_amount, 
        avg(sales_amount) over (partition by sales_date) 
FROM sales
ORDER BY 1
```
- added another avg(sales_amount) by sales date.



```sql
SELECT s.sales_date, 
       s.order_id, 
       s.product_id, 
       s.customer_id, 
       s.total_amount, 
       AVG (total_amount)  over () AS avg_total,
       AVG (total_amount)  over (partition by sales_date ) AS avg_by_day,
       AVG (total_amount)  over (partition by trunc(sales_date, 'mon' )) AS avg_by_month
FROM sales s
```
- 3 different average numbers were produced: total, daily, and monthly.



```sql
SELECT sales_date,
       order_id, 
       product_id, 
       sales_amount, 
       sum(sales_amount) over (order by sales_date, order_id, product_id ) as cum_sum
FROM sales
order by 1
```
- the cumulative sum of sales.



```sql
SELECT TRUNC(sales_date, 'MON') AS sales_month,
       SUM (total_amount) AS total_amount
       FROM sales 
GROUP BY TRUNC(sales_date, 'MON')
ORDER BY 1
```
- monthly sales



```sql
SELECT TRUNC(sales_date, 'MON') AS sales_month,
       SUM (sales_amount) AS sales_amount,
       RATIO_TO_REPORT (SUM(sales_amount)) OVER () AS RATIO
FROM sales s
GROUP BY TRUNC(sales_date, 'MON')
ORDER BY 1
```
- RATIO_TO_REPORT. Monthly sales against total sales.



```sql
SELECT TRUNC(sales_date, 'MON') AS sales_month,
       SUM (sales_amount) AS sales_amount,
ROUND(RATIO_TO_REPORT (SUM(sales_amount)) OVER (),2) AS RATIO
FROM sales s
GROUP BY TRUNC(sales_date, 'MON')
ORDER BY 1
```
- The coefficient is rounded.



```sql
SELECT TRUNC(sales_date, 'MON') AS sales_month,
       SUM (sales_amount) AS sales_amount,
       RATIO_TO_REPORT (SUM(sales_amount)) OVER () * 100 AS RATIO_PERCENT
FROM sales s
GROUP BY TRUNC(sales_date, 'MON')
ORDER BY 1
```
- monthly sales, aslo it's percentage from total sales



```sql
SELECT TRUNC(s.sales_date, 'mon') AS sales_month,
	sp.first_name,
	SUM (sales_amount) AS sales_amount,
	RANK ()
	OVER (PARTITION BY TRUNC(s.sales_date, 'mon') ORDER BY SUM(sales_amount) DESC) AS salesperson_rank_top
FROM sales s, salesperson sp
WHERE s.salesperson_id = sp.salesperson_id
GROUP BY TRUNC (s.sales_date, 'mon'), sp.first_name
```
- added new column with salespersons mothly ranking.



```sql
SELECT * FROM
(
SELECT TRUNC(s.sales_date, 'mon') AS sales_month,
	sp.first_name,
	SUM (sales_amount) AS sales_amount,
	RANK ()
	OVER (PARTITION BY TRUNC(s.sales_date, 'mon') ORDER BY SUM(sales_amount) DESC) AS RANK1
FROM sales s, salesperson sp
WHERE s.salesperson_id = sp.salesperson_id
GROUP BY TRUNC (s.sales_date, 'mon'), sp.first_name
)
WHERE RANK1 <= 3
```
- salespersons list and their Top-3 ranking.



```sql
SELECT  sp.first_name,
             SUM (sales_amount) AS sales_amount,	
             NTILE (3)	OVER (ORDER BY SUM (sales_amount) DESC) AS BAND
FROM sales s, salesperson sp
WHERE s.salesperson_id = sp.salesperson_id
GROUP BY sp.first_name
```
- we divided  salespersons into 3 groups, depending on their results.



```sql
SELECT sales_month,
sales_amount, previous_month,
ROUND(((sales_amount - previous_month) / previous_month) * 100,2) AS growth_percent
FROM
(
SELECT 	TRUNC (s.sales_date, 'mon') AS sales_month,
	SUM (sales_amount) AS sales_amount,
	LAG (SUM(sales_amount), 1) OVER (ORDER BY TRUNC(sales_date, 'mon')) AS previous_month
FROM sales s
GROUP BY TRUNC (s.sales_date, 'mon') 
)
```
- analyzing sales growth across time.